# 1 Transfer learning the last layer using the bottleneck as input

### to decide the further transfer learning models size

In [1]:
from __future__ import print_function, division
import argparse
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torch.nn import functional as F
import numpy as np
import torchvision
from torchvision import datasets, models, transforms, utils
import matplotlib.pyplot as plt
import time
import os
import shutil
import sys
import random
import errno
import seaborn as sns
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
from numpy import set_printoptions
import matplotlib.pyplot as plt
import pandas as pd
from save_resnet18_test_scores import predict_via_single_model
from ImageFolderData import ImageFolder
from Resnet101Layer4DinputData import leaf_resnet_eval_layer_tensor
from DataWDistilledLogits import leaf_resnet_train_distill
from get_result_file_paths import get_saved_logits_fn, get_result_file_paths
from ResNet101BaseModel import ResNet101FC
from ResNet101TransferModel import NewModelFromResNet101BottleNeck
from ResNet18Model import ResNet18FC
from resnet18_run import share_parser
from initial_transferlearn_models import resnet18Base, resnet34Base, resnet50Base, resnet101Base, resnet152Base, resnet18t, resnet34t, resnet50t, resnet101t, resnet152t
if sys.version_info[0] == 2:
    import cPickle as pickle
else:
    import pickle
%matplotlib inline

In [2]:
parser = share_parser()
args = parser.parse_args([])
args.dataseparate='50-50'
#https://stackoverflow.com/questions/41961949/google-oauth-inside-jupyter-notebook
assert args.train_size >= args.batch_size

In [3]:
args.cuda = not args.no_cuda and torch.cuda.is_available()
print(args.cuda)
print(args)

True
Namespace(T=3.5, arch='resnet18', batch_size=100, beta1=0.5, cuda=True, dataroot='/home/h/Downloads/plantvillage_deeplearning_paper_dataset', dataseparate='50-50', dataset='leaf', datasettype='train', distill='False', dr=0.5, epochs=30, eta=20.0, has_weights='True', lr=0.0002, no_cuda=False, num_channels=3, num_hidden_neuron=64, resume='', rlr=0.003, saveroot='/media/h/15210519917/resnet18', seed=5306, softlogits='False', start_epoch=0, test_batch_size=100, train_size=100, transferinputdir='bottleneck_tensors_labels/5306seed_layer3', wd=0.0005)


In [4]:
if args.seed is None:
    args.seed = random.randint(1, 10000)
print("Random Seed: ", args.seed)
random.seed(args.seed)
torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed_all(args.seed)
    torch.backends.cudnn.benchmark = True

Random Seed:  5306


In [5]:
kwargs = {'num_workers': 2, 'pin_memory': True} if args.cuda else {}
data_separate = args.dataseparate

In [6]:
def predict(arch):
    if arch == 'resnet18':
        transfer_model = resnet18t()
    elif arch == 'resnet34':
        transfer_model = resnet34t()
    elif arch == 'resnet50':
        transfer_model = resnet50t()
    elif arch == 'resnet101':
        transfer_model = resnet101t()
    elif arch == 'resnet152':
        transfer_model = resnet152t()

    CE_loss = nn.CrossEntropyLoss()

    if args.cuda:
        transfer_model = torch.nn.DataParallel(transfer_model).cuda()
        CE_loss = CE_loss.cuda()
    model_dir = os.path.join('/media/h/15210519917','initial_tl', arch)
    model_path = os.path.join(model_dir, data_separate+'_'+str(29)+'.pth.tar')
    preds_labels_file = os.path.join(model_dir, data_separate+'_'+str(29)+'.pt')
    leaf_test_loader =  torch.utils.data.DataLoader(
        leaf_resnet_eval_layer_tensor(root_dir = args.dataroot
                                      , data_type='test',data_separate=data_separate
                                      ,transf_in_dir='bottleneck_tensors_labels/'+arch
                                     ), 
        batch_size=args.test_batch_size, shuffle=False, **kwargs)
    if os.path.exists(preds_labels_file) == False:
            if os.path.isfile(model_path):
                print("=> loading checkpoint '{}'".format(model_path))
                checkpoint = torch.load(model_path)
                transfer_model.load_state_dict(checkpoint['state_dict'])
                print("=> loaded checkpoint '{}' (epoch {})"
                      .format(model_path, checkpoint['epoch']))
            else:
                print("=> no checkpoint found at '{}'".format(model_path))

    transfer_model.eval()

    predict_via_single_model(leaf_test_loader, transfer_model, CE_loss, args.test_batch_size, preds_labels_file=preds_labels_file, cuda=args.cuda)

In [7]:
for arch in ['resnet18']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet18/20-80_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet18/20-80_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet18/20-80_29.pt
create finished


In [8]:
for arch in ['resnet18']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

datasize: 43538
reading /media/h/15210519917/initial_tl/resnet18/20-80_29.pt
torch.Size([43538, 38])
torch.Size([43538])
0.19011782109737396
Acc: 0.94093 [40966/43538], f1: 0.94005, precision: 0.94305, recall: 0.94093, loss: 0.19012


In [9]:
for arch in ['resnet34']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet34/20-80_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet34/20-80_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet34/20-80_29.pt
create finished


In [10]:
for arch in ['resnet34']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

datasize: 43538
reading /media/h/15210519917/initial_tl/resnet34/20-80_29.pt
torch.Size([43538, 38])
torch.Size([43538])
0.20635737478733063
Acc: 0.93544 [40727/43538], f1: 0.93564, precision: 0.94082, recall: 0.93544, loss: 0.20636


In [11]:
for arch in ['resnet50']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet50/20-80_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet50/20-80_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet50/20-80_29.pt
create finished


In [12]:
for arch in ['resnet50']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

datasize: 43538
reading /media/h/15210519917/initial_tl/resnet50/20-80_29.pt
torch.Size([43538, 38])
torch.Size([43538])
0.16790658235549927
Acc: 0.94899 [41317/43538], f1: 0.94940, precision: 0.95297, recall: 0.94899, loss: 0.16791


In [16]:
for arch in ['resnet101']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet101/20-80_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet101/20-80_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet101/20-80_29.pt
create finished


In [17]:
for arch in ['resnet101']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

datasize: 43538
reading /media/h/15210519917/initial_tl/resnet101/20-80_29.pt
torch.Size([43538, 38])
torch.Size([43538])
0.18912293016910553
Acc: 0.94462 [41127/43538], f1: 0.94382, precision: 0.95044, recall: 0.94462, loss: 0.18912


In [18]:
for arch in ['resnet152']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet152/20-80_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet152/20-80_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet152/20-80_29.pt
create finished


In [19]:
for arch in ['resnet152']: #'resnet18', 'resnet34', 'resnet50', 'resnet101', 
    args.arch = arch
    predict(arch)

datasize: 43538
reading /media/h/15210519917/initial_tl/resnet152/20-80_29.pt
torch.Size([43538, 38])
torch.Size([43538])
0.20366032421588898
Acc: 0.94221 [41022/43538], f1: 0.93851, precision: 0.94504, recall: 0.94221, loss: 0.20366


In [7]:
for arch in ['resnet18', 'resnet34', 'resnet50', 'resnet101','resnet152']: #
    args.arch = arch
    predict(arch)

=> loading checkpoint '/media/h/15210519917/initial_tl/resnet18/50-50_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet18/50-50_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet18/50-50_29.pt
create finished
=> loading checkpoint '/media/h/15210519917/initial_tl/resnet34/50-50_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet34/50-50_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet34/50-50_29.pt
create finished
=> loading checkpoint '/media/h/15210519917/initial_tl/resnet50/50-50_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet50/50-50_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet50/50-50_29.pt
create finished
=> loading checkpoint '/media/h/15210519917/initial_tl/resnet101/50-50_29.pth.tar'
=> loaded checkpoint '/media/h/15210519917/initial_tl/resnet101/50-50_29.pth.tar' (epoch 30)
creating /media/h/15210519917/initial_tl/resnet101/50-50_29.pt
creat

In [8]:
for arch in ['resnet18', 'resnet34', 'resnet50', 'resnet101','resnet152']: #
    args.arch = arch
    predict(arch)

datasize: 27311
reading /media/h/15210519917/initial_tl/resnet18/50-50_29.pt
torch.Size([27311, 38])
torch.Size([27311])
0.15847019851207733
Acc: 0.95141 [25984/27311], f1: 0.95133, precision: 0.95341, recall: 0.95141, loss: 0.15847
datasize: 27311
reading /media/h/15210519917/initial_tl/resnet34/50-50_29.pt
torch.Size([27311, 38])
torch.Size([27311])
0.2021794319152832
Acc: 0.93633 [25572/27311], f1: 0.93904, precision: 0.94806, recall: 0.93633, loss: 0.20218
datasize: 27311
reading /media/h/15210519917/initial_tl/resnet50/50-50_29.pt
torch.Size([27311, 38])
torch.Size([27311])
0.14000412821769714
Acc: 0.95723 [26143/27311], f1: 0.95868, precision: 0.96397, recall: 0.95723, loss: 0.14000
datasize: 27311
reading /media/h/15210519917/initial_tl/resnet101/50-50_29.pt
torch.Size([27311, 38])
torch.Size([27311])
0.14586341381072998
Acc: 0.95548 [26095/27311], f1: 0.95589, precision: 0.95950, recall: 0.95548, loss: 0.14586
datasize: 27311
reading /media/h/15210519917/initial_tl/resnet152/50